In [1]:
from pathlib import Path
import itertools
import numpy as np
import matplotlib.pylab as plt
# %matplotlib tk
# %matplotlib nbagg
# %matplotlib inline
import h5py
import json
import pickle

## Loading files

In [2]:
filename_1 = Path("C:/LINHC/VersucheDBs/Trelleborg/2021-01-27-V24/2_Phase_A-D/20210127_Phase_A-D_LEM1.h5")
# filename_2 = Path("C:/LINHC/VersucheDBs/Trelleborg/2021-01-27-V24/2_Phase_A-D/20210127_Phase_A-D_LEM2.h5")

h51 = h5py.File(filename_1, "r")
# h52 = h5py.File(filename_2, "r")

print("ATTR:")
print(h51.attrs.keys())

print("Keys:")
print(h51.keys())



ATTR:
<KeysViewHDF5 ['name_lookup', 'name_rev_lookup']>
Keys:
<KeysViewHDF5 ['S11_LEM1', 'S21_P1_LEM1', 'S21_P2_LEM1', 'S22_LEM1', 'hub1', 'hub2', 'ip1k1', 'ip1k2', 'ip2k1', 'ip2k2', 'k1at', 'k1f', 'k1p', 'k1t', 'k2at', 'k2f', 'k2p', 'k2t', 'phi', 'pos1', 'pos2', 'shub1', 'shub2', 'sk1at', 'sk1f', 'sk1p', 'sk1t', 'sk2at', 'sk2f', 'sk2p', 'sk2t', 'spd1', 'spd2', 'sphi', 'spos1', 'spos2', 'sspd1', 'sspd2', 'stdf', 't', 'tdf']>


In [10]:
shift_opts_freqs = pickle.load( open( "shift_opts_freqs.pkl", "rb" ) )

In [3]:
json.loads(h51.attrs["name_lookup"])

{'Ist-Ablauf-Zeit, ca.': 't',
 'Ist-MG1: Teller-Drehfrequenz [Hz]': 'tdf',
 'Soll-MG1: Teller-Drehfrequenz [Hz]': 'stdf',
 'Ist-MG2: Kammer1-Temp[°C]': 'k1t',
 'Soll-MG2: Kammer1-Temp[°C]': 'sk1t',
 'Ist-MG3: Kammer1-Druck[bar]': 'k1p',
 'Soll-MG3: Kammer1-Druck[bar]': 'sk1p',
 'Ist-MG4: Kammer2-Temp.[°C]': 'k2t',
 'Soll-MG4: Kammer2-Temp.[°C]': 'sk2t',
 'Ist-MG5: Kammer2-Druck[bar]': 'k2p',
 'Soll-MG5: Kammer2-Druck[bar]': 'sk2p',
 'Ist-MG6: Kammer1-Kraft[kN]': 'k1f',
 'Soll-MG6: Kammer1-Kraft[kN]': 'sk1f',
 'Ist-MG7: Kammer2-Kraft[kN]': 'k2f',
 'Soll-MG7: Kammer2-Kraft[kN]': 'sk2f',
 'Ist-MG8: Kammer1-Außentemp.[°C]': 'k1at',
 'Soll-MG8: Kammer1-Außentemp.[°C]': 'sk1at',
 'Ist-MG9: Kammer2-Außentemp.[°C]': 'k2at',
 'Soll-MG9: Kammer2-Außentemp.[°C]': 'sk2at',
 'Ist-MG10: Kammer1-Istpos.[mm]': 'pos1',
 'Soll-MG10: Kammer1-Istpos.[mm]': 'spos1',
 'Ist-MG11: Kammer2-Istpos.[mm]': 'pos2',
 'Soll-MG11: Kammer2-Istpos.[mm]': 'spos2',
 'Ist-MG12: Hub1[mm]': 'hub1',
 'Soll-MG12: Hub1[mm]': '

## Find the low speed areas

all low speed areas (low)

In [5]:
low = h51['spd1'][:]>-50
pos_low = h51['pos1'][low]
s21_low = h51['S21_P1_LEM1'][low,fidx]

all low speed areas as ONE sequence

In [6]:
low = np.where(h51['spd1'][:]>-50)[0]
pos_low = h51['pos1'][low]
s21_low = h51['S21_P1_LEM1'][low,fidx]

all low speed areas as seperate arrays

In [7]:
low_sep = np.where(np.diff(low)>10000)[0]
pos_low_sep = np.split(pos_low,low_sep+1)
s21_low_sep = np.split(s21_low,low_sep+1)

cut out some points to make sure all points in low speed 
(if not some points in high speed area will showed up in s21, especially in later timestamps)

In [46]:
Ls = np.append(low[low_sep],low[-1])
print("timestamps of Ls:")
print (Ls)

Fs = np.append(low[0],low[low_sep+1])
print("timestamps of Fs:")
print (Fs)

print("Durations of lsa:\n",Ls-Fs)

timestamps of Ls:
[ 14443  35899  57359  78815 100274 121732 143192 164649 186106 207561
 229019 250475 271931 293390 314847 336304 357760 379214 400670 422129
 443587 465047 486506 507966 529422 550879 572338 593794 615254 636714
 658168 679621 701081 722537 743992 765451 786909 808367 829825 851284
 872738 894197 915655 937110 958570 980026]
timestamps of Fs:
[ 11670  33126  54583  76043  97498 118957 140416 161876 183333 204790
 226245 247702 269159 290615 312073 333530 354987 376444 397897 419354
 440813 462271 483731 505189 526650 548106 569563 591022 612478 633938
 655398 676852 698305 719764 741221 762676 784135 805593 827050 848509
 869967 891422 912880 934339 955794 977254]
Durations of lsa:
 [2773 2773 2776 2772 2776 2775 2776 2773 2773 2771 2774 2773 2772 2775
 2774 2774 2773 2770 2773 2775 2774 2776 2775 2777 2772 2773 2775 2772
 2776 2776 2770 2769 2776 2773 2771 2775 2774 2774 2775 2775 2771 2775
 2775 2771 2776 2772]


find the max values of positions in each low speed interval

In [54]:
# create a list of arrays max_sep, each array contains peak values of the corresponding low speed area.
max_sep = [] # (46,4)
for k in range(len(pos_low_sep)):
    # temporary sequency ts
    ts = pos_low_sep[k] 

    # find top 30 highest values
    max_20 = np.argsort(ts)[-30:][::-1]

    # filter the max value for each period
    temp =[max_20[0]]
    for i in max_20:
        if all([abs(j-i)>50 for j in temp]):        
            temp.append(i)
    max_sep.append(np.sort(temp))
    
# There should be 4 max values in each interval
assert all([len(max_sep[i])==4 for i in range(len(max_sep))]) 

## Data extraction

In [11]:
# calculated by grid search
shift_opts_freqs.shape

(121, 46)

In [65]:
# # choose intervals with safe points (2000 points after the first peak value, 
# 2000 is to make sure no points in hsa are included, peak value is to make sure the same starting point for each interval)

pos=[]
s21=[]
# tdf=[] # same as spd1?
k1t=[]
k1p=[]
k1f=[]
k1at=[]
# hub1=[] # constant
phi=[]
spd1=[]
ip1k1=[]
ip2k1=[]

for i,Fc in enumerate(Fs):
    F = Fc + max_sep[i][0] 
    L = F + 2000
#     FF = [F for _ - shift_opts[i]
#     LL = L - shift_opts[i]
          
#     s21.append(h51['S21_P1_LEM1'][FF:LL, :])
    pos.append(h51['pos1'][F:L])
    # tdf.append(h51['tdf'][F:L])
    k1p=append(h51['k1p'][F:L])
    k1f=append(h51['k1f'][F:L])
    k1at=append(h51['k1at'][F:L])
    # hub1=append(h51['hub1'][F:L]) 
    phi=append(h51['phi'][F:L])
    spd1=append(h51['spd1'][F:L])
    ip1k1=append(h51['ip1k1'][F:L])
    ip2k1=append(h51['ip2k1'][F:L])

## Data preprocessing

normalize? imaginary number? 

## Training

### NN

### RF